12. Kernel PCA

    Objetivo: Kernel PCA é uma extensão do PCA que usa funções kernel para mapear os dados em um espaço de maior dimensão, permitindo capturar relações não lineares.

    Uso: Usado quando os dados não são linearmente separáveis e se deseja capturar complexas interações entre os dados.


In [1]:
#KERNEL PCA


import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.decomposition import KernelPCA
import seaborn as sns

# Diretório das imagens
base_dir = './png'
image_size = (128, 128)  # Ajuste conforme o tamanho das imagens de galáxias
image_shape = np.prod(image_size)  # Número total de pixels por imagem

# Inicializar a lista para armazenar as imagens
image_list = []

# Obter todos os arquivos de imagem no diretório
image_files = [f for f in os.listdir(base_dir) if f.endswith('.png')]  # Ajuste a extensão conforme necessário

# Carregar e processar cada imagem
for image_name in image_files:
    img_path = os.path.join(base_dir, image_name)
    img = Image.open(img_path).convert('L')  # Converte a imagem para escala de cinza
    img = img.resize(image_size)  # Redimensiona a imagem, se necessário
    img_array = np.array(img).flatten()  # Achata a imagem
    image_list.append(img_array)

# Verificar se a lista de imagens está vazia
if not image_list:
    raise ValueError("Nenhuma imagem foi carregada. Verifique o diretório e os arquivos.")

# Convertendo a lista para um array numpy
X = np.array(image_list)

# Verificar se X tem pelo menos 2 dimensões
if len(X.shape) != 2:
    raise ValueError("O array X não tem o formato esperado. Verifique o carregamento das imagens.")

# Aplicar Kernel PCA
n_components = min(X.shape[0], X.shape[1])  # Número máximo de componentes que podemos calcular
kpca = KernelPCA(n_components=n_components, kernel='rbf', gamma=10, fit_inverse_transform=True)
X_kpca = kpca.fit_transform(X)
X_reconstructed = kpca.inverse_transform(X_kpca)  # Reconstrução das imagens

# Plotar a projeção no espaço Kernel PCA (Componente 1 vs Componente 2)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_kpca[:, 0], y=X_kpca[:, 1], alpha=0.5)
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.title("Projeção das Imagens no Espaço Kernel PCA")
plt.show()

# Plotar as primeiras imagens reconstruídas
fig, axes = plt.subplots(5, 5, figsize=(10, 10))  # Grid 5x5 para visualização das reconstruções
for i, ax in enumerate(axes.flat):
    if i < n_components:
        component = X_reconstructed[i].reshape(128, 128)  # Ajuste para o tamanho original da imagem
        ax.imshow(component, cmap='gray')
        ax.set_title(f"Reconstrução {i+1}")
        ax.axis("off")

plt.tight_layout()
plt.show()


# Gráfico de erro da reconstrução
erro_reconstrucao = np.mean((X - X_reconstructed) ** 2, axis=1)
plt.figure(figsize=(8, 6))
sns.histplot(erro_reconstrucao, bins=20, color='red', kde=True, linewidth=1.5)
plt.xlabel("Erro de Reconstrução", fontsize=12)
plt.ylabel("Frequência", fontsize=12)
plt.title("Distribuição do Erro de Reconstrução das Imagens", fontsize=14)
plt.tight_layout()
plt.show()

